<center>
# ПРИЛОЖЕНИЕ 5
<center>
# РЕШЕНИЕ ДОМАШНЕГО ЗАДАНИЯ 1

# Этап I. Построение модели логистической регрессии на обучающей выборке - части исторической выборки и ее проверка на контрольной выборке - части исторической выборки с помощью класса LogisticRegression

In [1]:
# импортируем необходимые библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), позволяющую выполнить
# разбиение на обучающую и контрольную выборки
from sklearn.model_selection import train_test_split
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer
# импортируем класс PowerTransformer, позволяющий выполнить 
# преобразование Бокса-Кокса/Йео-Джонсона и стандартизацию
from sklearn.preprocessing import PowerTransformer
# импортируем класс OneHotEncoder, позволяющий
# выполнить дамми-кодирование
from sklearn.preprocessing import OneHotEncoder
# импортируем класс KBinsDiscretizer, 
# позволяющий выполнять биннинг
from sklearn.preprocessing import KBinsDiscretizer
# импортируем класс ColumnTransformer, позволяющий выполнять
# преобразования для отдельных типов столбцов
from sklearn.compose import ColumnTransformer
# импортируем класс Pipeline, позволяющий создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс KFold и функцию cross_val_score
# для выполнения перекрестной проверки
from sklearn.model_selection import KFold, cross_val_score
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression
# импортируем функцию roc_auc_score()
from sklearn.metrics import roc_auc_score
# импортируем класс GridSearchCV
from sklearn.model_selection import GridSearchCV

## I.1. Загрузка исторических данных

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Churn_logreg.csv', 
                   encoding='cp1251', sep=';')

## I.2. Вывод общей информации о датафрейме

In [3]:
# выводим первые 5 наблюдений датафрейма
data.head(10)

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn
0,"27,09",0,"39,74",Нет,Бюджетный,CC,35.0,Женский,Женат,0.0,77680,Остается
1,NaN,0,"46,31",Нет,NaN,NaN,53.0,Мужской,Одинокий,1.0,"37111,5",Остается
2,"23,76",0,NaN,NaN,Бюджетный,Auto,NaN,Женский,NaN,1.0,"37079,4",Остается
3,"9,4",NaN,"13,9",Нет,NaN,CH,NaN,Мужской,Одинокий,NaN,81997,Остается
4,"14,15",0,"108,43",Да,Бесплатный,Auto,39.0,Женский,Одинокий,0.0,"16829,6",Остается
5,"7,68",NaN,NaN,Нет,NaN,CC,57.0,NaN,NaN,1.0,"57272,7",Остается
6,"27,37","0,2","48,9",Да,Бесплатный,CH,NaN,Женский&*,NaN,1.0,NaN,Уходит
7,"5,35",NaN,NaN,Да,NaN,NaN,55.0,Женский&*,Одинокий,0.0,"40200,7",Уходит
8,"14,77",0,"30,42",Нет,Бесплатный,CH,86.0,Мужской,NaN,0.0,"92167,3",Остается
9,"24,73",NaN,"21,17",Нет,Бюджетный,CC,28.0,Мужской,Одинокий,0.0,"37135,5",Уходит


## I.3. Изменение типов переменных

In [4]:
# смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4431 entries, 0 to 4430
Data columns (total 12 columns):
longdist    4430 non-null object
internat    4427 non-null object
local       4428 non-null object
int_disc    4430 non-null object
billtype    4427 non-null object
pay         4429 non-null object
age         4428 non-null float64
gender      4430 non-null object
marital     4427 non-null object
children    4430 non-null float64
income      4430 non-null object
churn       4431 non-null object
dtypes: float64(2), object(10)
memory usage: 415.5+ KB


In [5]:
# заменяем запятые на точки и преобразуем в тип float
for i in ['longdist', 'internat', 'local', 'income']:
    data[i] = data[i].str.replace(',', '.').astype('float')
# заменяем строковые значения в переменной churn на числовые
data['churn'] = np.where(data['churn'] == 'Остается', 0, 1)
data.head(10)

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn
0,27.09,0.0,39.74,Нет,Бюджетный,CC,35.0,Женский,Женат,0.0,77680.0,0
1,NaN,0.0,46.31,Нет,NaN,NaN,53.0,Мужской,Одинокий,1.0,37111.5,0
2,23.76,0.0,NaN,NaN,Бюджетный,Auto,NaN,Женский,NaN,1.0,37079.4,0
3,9.40,NaN,13.90,Нет,NaN,CH,NaN,Мужской,Одинокий,NaN,81997.0,0
4,14.15,0.0,108.43,Да,Бесплатный,Auto,39.0,Женский,Одинокий,0.0,16829.6,0
5,7.68,NaN,NaN,Нет,NaN,CC,57.0,NaN,NaN,1.0,57272.7,0
6,27.37,0.2,48.90,Да,Бесплатный,CH,NaN,Женский&*,NaN,1.0,NaN,1
7,5.35,NaN,NaN,Да,NaN,NaN,55.0,Женский&*,Одинокий,0.0,40200.7,1
8,14.77,0.0,30.42,Нет,Бесплатный,CH,86.0,Мужской,NaN,0.0,92167.3,0
9,24.73,NaN,21.17,Нет,Бюджетный,CC,28.0,Мужской,Одинокий,0.0,37135.5,1


In [6]:
# смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4431 entries, 0 to 4430
Data columns (total 12 columns):
longdist    4430 non-null float64
internat    4427 non-null float64
local       4428 non-null float64
int_disc    4430 non-null object
billtype    4427 non-null object
pay         4429 non-null object
age         4428 non-null float64
gender      4430 non-null object
marital     4427 non-null object
children    4430 non-null float64
income      4430 non-null float64
churn       4431 non-null int64
dtypes: float64(6), int64(1), object(5)
memory usage: 415.5+ KB


## I.4. Получение подробной информации о количественных и категориальных переменных

In [7]:
# смотрим статистики для количественных переменных
data.describe()

,longdist,internat,local,age,children,income,churn
count,4430.000000,4427.000000,4428.000000,4428.000000,4430.000000,4430.000000,4431.000000
mean,13.638023,0.835044,51.361093,57.562331,0.988939,50296.423896,0.436696
std,9.393025,2.233639,54.685012,22.832404,0.824448,28439.435264,0.496032
min,0.000000,0.000000,0.050000,18.000000,0.000000,110.280000,0.000000
25%,5.170000,0.000000,13.460000,38.000000,0.000000,25445.600000,0.000000
50%,13.680000,0.000000,35.010000,58.000000,1.000000,50290.700000,0.000000
75%,22.080000,0.000000,71.660000,77.000000,2.000000,75004.500000,1.000000
max,29.980000,9.950000,450.620000,97.000000,2.000000,99832.900000,1.000000


In [8]:
# смотрим статистики для категориальных переменных,
# кроме зависимой переменной churn, создав список
# категориальных переменных
categorical_columns = [c for c in data.loc[:, data.columns != 'churn'] if data[c].dtype.name == 'object']
data[categorical_columns].describe()

,int_disc,billtype,pay,gender,marital
count,4430,4427,4429,4430,4427
unique,2,2,4,4,5
top,Нет,Бюджетный,CC,Женский,Женат
freq,3054,2244,2561,2240,2620


In [9]:
# смотрим уникальные значения
# категориальных переменных
for c in categorical_columns:
    print(data[c].unique())

['Нет' nan 'Да']
['Бюджетный' nan 'Бесплатный']
['CC' nan 'Auto' 'CH' 'CD']
['Женский' 'Мужской' nan 'Женский&*' 'Мужской&*']
['Женат' 'Одинокий' nan '_Одинокий' '_Женат' 'Же&нат']


In [10]:
# смотрим частоты категорий для
# категориальных переменных
for c in categorical_columns:
    print(data[c].value_counts(dropna=False))

Нет    3054
Да     1376
NaN       1
Name: int_disc, dtype: int64
Бюджетный     2244
Бесплатный    2183
NaN              4
Name: billtype, dtype: int64
CC      2561
CH       977
Auto     889
CD         2
NaN        2
Name: pay, dtype: int64
Женский      2240
Мужской      2183
Женский&*       4
Мужской&*       3
NaN             1
Name: gender, dtype: int64
Женат        2620
Одинокий     1800
NaN             4
_Женат          3
Же&нат          2
_Одинокий       2
Name: marital, dtype: int64


## I.5. Нормализация строковых значений и обработка редких категорий

In [11]:
# удаляем лишние символы в категориях переменных
# gender и marital
for i in ['gender', 'marital']:
    data[i] = data[i].str.replace('[*&_]', '')

# смотрим результаты
for i in ['gender', 'marital']:
    print(data[i].value_counts(dropna=False))

Женский    2244
Мужской    2186
NaN           1
Name: gender, dtype: int64
Женат       2625
Одинокий    1802
NaN            4
Name: marital, dtype: int64


In [12]:
# заменяем редкую категорию модой
data.at[data['pay'] == 'CD', 'pay'] = 'CC'
data['pay'].value_counts(dropna=False)

CC      2563
CH       977
Auto     889
NaN        2
Name: pay, dtype: int64

## I.6. Конструирование признаков

In [13]:
# пишем функцию, создающую парные взаимодействия
def make_conj(df, feature1, feature2):
    df[feature1 + "_" + feature2] = df[feature1].astype('object') + " + " + df[feature2].astype('object')

In [14]:
# применяем функцию
make_conj(data, 'gender', 'marital')

In [15]:
# поделим возраст на длительность междугородних звонков в минутах
data['ratio'] = data['age'] / data['longdist']
# заменяем бесконечные значения на 1
data['ratio'].replace([np.inf, -np.inf], 1, inplace=True)

In [16]:
# поделим доход на возраст
data['ratio2'] = data['income'] / data['age']
# заменяем бесконечные значения на 0
data['ratio2'].replace([np.inf, -np.inf], 0, inplace=True)

In [17]:
# поделим возраст на количество детей
data['ratio3'] = data['age'] / data['children']
# заменяем бесконечные значения на 0
data['ratio3'].replace([np.inf, -np.inf], 0, inplace=True)

In [18]:
# поделим длительность междугородних звонков в минутах на
# длительность международных звонков в минутах
data['ratio4'] = data['longdist'] / data['internat']
# заменяем бесконечные значения на 0
data['ratio4'].replace([np.inf, -np.inf], 0, inplace=True)

## I.7. Разбиение набора данных на обучающую и контрольную выборки

In [19]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
train, test, y_train, y_test = train_test_split(data.drop('churn', axis=1), 
                                                data['churn'], 
                                                test_size=.3, 
                                                stratify=data['churn'], 
                                                random_state=100)

In [20]:
# выводим информацию о количестве пропусков
# по каждой переменной в полученных выборках
print(train.isnull().sum())
print(test.isnull().sum())

longdist            1
internat            3
local               2
int_disc            0
billtype            3
pay                 2
age                 1
gender              1
marital             2
children            0
income              1
gender_marital      2
ratio               2
ratio2              1
ratio3              1
ratio4            357
dtype: int64
longdist            0
internat            1
local               1
int_disc            1
billtype            1
pay                 0
age                 2
gender              0
marital             2
children            1
income              0
gender_marital      2
ratio               2
ratio2              2
ratio3              2
ratio4            140
dtype: int64


In [21]:
from sklearn.base import TransformerMixin
class CustomBinning(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        bins = np.array([-np.inf,  0.14, 8.81, np.inf])
        X_bin = np.digitize(X, bins)
        return X_bin

In [22]:
class Replacer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X==0, 0.5, X) 
        return X_replaced

In [23]:
# создаем списки количественных 
# и категориальных столбцов
cat_columns = train.dtypes[train.dtypes == 'object'].index
num_columns = train.dtypes[train.dtypes != 'object'].index
num_columns

Index(['longdist', 'internat', 'local', 'age', 'children', 'income', 'ratio',
       'ratio2', 'ratio3', 'ratio4'],
      dtype='object')

In [24]:
num_columns_for_bin = ['longdist']
num_columns_for_kbindisc = ['local', 'age']

In [25]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('replacer', Replacer()),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True))
])

num_bin_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('replacer', Replacer()),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True)),
    ('bin', CustomBinning()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

num_kbindisc_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('replacer', Replacer()),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True)),
    ('kbd', KBinsDiscretizer(n_bins=3, encode='onehot-dense'))
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [26]:
transformers = [('num', num_pipe, num_columns),
                ('num_bin', num_bin_pipe, num_columns_for_bin),
                ('num_kbindisc', num_kbindisc_pipe, num_columns_for_kbindisc),
                ('cat', cat_pipe, cat_columns)
               ]
ct = ColumnTransformer(transformers=transformers)
ct.fit(train)
X_train = ct.transform(train)
X_test = ct.transform(test)

In [27]:
logreg = LogisticRegression(solver='lbfgs', max_iter=200).fit(X_train, y_train)
# печатаем значения AUC
print("AUC на обучающей выборке: {:.3f}".format(
    roc_auc_score(y_train, logreg.predict_proba(X_train)[:, 1])))
print("AUC на контрольной выборке: {:.3f}".format(
    roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1])))

AUC на обучающей выборке: 0.893
AUC на контрольной выборке: 0.890


In [28]:
X = data
y = data.pop('churn').values

In [29]:
# создаем экземпляр класса KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
ml_pipe = Pipeline([('transform', ct), ('logreg', LogisticRegression(
    solver='lbfgs', max_iter=200))])
# выполняем перекрестную проверку, конвейер размещен
# внутри цикла перекрестной проверки
cross_val_score(ml_pipe, X, y, scoring='roc_auc', cv=kf).mean()

0.8904699746025904

In [30]:
# задаем сетку гиперпараметров
param_grid = {
    'logreg__C': [0.02, 0.04, 0.08, 0.2, 0.4, 0.6, 0.8, 1],
    'transform__num_kbindisc__kbd__n_bins': [2, 3, 4],
    'transform__num_kbindisc__kbd__strategy': ['uniform', 
                                               'quantile', 
                                               'kmeans'],
}
# передаем наш конвейер в объект GridSearchCV
gs = GridSearchCV(ml_pipe, param_grid,  scoring='roc_auc', cv=kf)
# выполняем решетчатый поиск
gs.fit(X, y)
# смотрим наилучшие значения гиперпараметров
gs.best_params_

{'logreg__C': 0.4,
 'transform__num_kbindisc__kbd__n_bins': 3,
 'transform__num_kbindisc__kbd__strategy': 'uniform'}

In [31]:
# смотрим наилучшее значение AUC
gs.best_score_

0.8925539965951182

In [32]:
# записываем CSV-файл в объект DataFrame 
fulldata = pd.read_csv('Data/Churn_logreg.csv', encoding='cp1251', sep=';')

In [33]:
# создаем массив меток и массив признаков
y_fulldata = fulldata.pop('churn').values

In [34]:
# пишем функцию, выполняющую предварительную обработку исторических данных
def preprocessing(df):
    # заменяем запятые на точки и преобразуем в тип float
    for i in ['longdist', 'internat', 'local', 'income']:
        df[i] = df[i].str.replace(',', '.').astype('float')
    # удаляем возможные лишние символы (все символы, не являющиеся 
    # буквами, символы нижнего подчеркивания и цифры) в категориях 
    # переменных gender и marital
    for i in ['gender', 'marital']:
        df[i] = df[i].str.replace('[\d+\W_]', '')
    # все новые категории переменной pay заменяем модой
    replace_new_values = lambda s: 'CC' if s not in ['CC', 'Auto', 'CH', np.NaN] else s
    df['pay'] = df['pay'].map(replace_new_values)
    # создаем переменную - результат конъюнкции
    df['gender_marital'] = df['gender'].astype('object') + " + " + df['marital'].astype('object')
    # поделим возраст на длительность междугородних звонков в минутах
    df['ratio'] = df['age'] / df['longdist']
    # заменяем бесконечные значения на 1
    df['ratio'].replace([np.inf, -np.inf], 1, inplace=True)
    # поделим длительность междугородних звонков в минутах на
    # длительность международных звонков в минутах
    df['ratio2'] = df['longdist'] / df['internat']
    # заменяем бесконечные значения на 0
    df['ratio2'].replace([np.inf, -np.inf], 0, inplace=True)
    # поделим доход на возраст
    df['ratio3'] = df['income'] / df['age']
    # заменяем бесконечные значения на 0
    df['ratio3'].replace([np.inf, -np.inf], 0, inplace=True)
    # поделим возраст на количество детей
    df['ratio4'] = df['age'] / df['children']
    # заменяем бесконечные значения на 0
    df['ratio4'].replace([np.inf, -np.inf], 0, inplace=True)

In [35]:
# применяем функцию предварительной обработки 
# ко всем историческим данным
preprocessing(fulldata)

In [36]:
num_kbindisc_pipe_upd = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('replacer', Replacer()),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True)),
    ('kbd_upd', KBinsDiscretizer(n_bins=3, strategy='uniform', encode='onehot-dense'))
])

In [37]:
# создаем список трансформеров
transformers_full = [('num', num_pipe, num_columns),
                     ('num_bin', num_bin_pipe, num_columns_for_bin),
                     ('num_kbindisc', num_kbindisc_pipe_upd, num_columns_for_kbindisc),
                     ('cat', cat_pipe, cat_columns)
                    ]
ct_full = ColumnTransformer(transformers=transformers_full)

In [38]:
fulldata = ct_full.fit_transform(fulldata)

In [39]:
# строим модель логистической регрессии на всех исторических данных
logreg_full = LogisticRegression(solver='lbfgs', C=0.4, max_iter=200).fit(fulldata, y_fulldata)
# печатаем значения AUC
print("AUC на всей исторической выборке: {:.3f}".format(
    roc_auc_score(y_fulldata, logreg_full.predict_proba(fulldata)[:, 1])))

AUC на всей исторической выборке: 0.896


In [40]:
# записываем CSV-файл, содержащий новые данные,
# в объект DataFrame
newdata = pd.read_csv('Data/Churn_new.csv', 
                      encoding='utf-8', sep=';')

In [41]:
# применяем функцию предварительной обработки 
# к новым данным
preprocessing(newdata)

In [42]:
# применяем конвейеры к новым данным
newdata = ct_full.fit_transform(newdata)

In [43]:
# вычисляем вероятности классов для новых данных
prob = logreg_full.predict_proba(newdata)
# выведем вероятности для первых 5 наблюдений
prob[:5]

array([[0.07851663, 0.92148337],
       [0.11812079, 0.88187921],
       [0.84523635, 0.15476365],
       [0.83759179, 0.16240821],
       [0.01561342, 0.98438658]])